### Step 0: Set up environment

In [1]:
import pandas as pd
import pprint
import requests
from bs4 import BeautifulSoup


In [18]:
excelPath = '/Users/yenshuohsu/DDSC/MavenWebScrapping/永豐.保管系統.Lib元件版本清單.20240630.xlsx'

### Step 1: Format excel sheet to the desired format
| Library | Vulnerabilities | Date | Url |
| ------- | --------------- | ---- | --- |

In [17]:
# Read in the excel file
df = pd.read_excel(excelPath,sheet_name=0)
print(str(df.shape[0]) + " rows, " + str(df.shape[1]) + " columns")

95 rows, 12 columns


In [10]:
# Convert to a list of dictionary to iterate through
# Assuming format is consistent. We're interested in Column G - L (7 - 12)
cropped = df.iloc[1:, 5:12] # 0 based, first row is header

libraries = cropped.iloc[:, 0]
vulnerabilities = cropped.iloc[:,1]
date= cropped.iloc[:, 3]
url = cropped.iloc[:, 5]

dependencies = []

for i in range(libraries.shape[0]):
    dependencies.append({
        "library": libraries.iloc[i],
        "vulnerabilities": vulnerabilities.iloc[i],
        "date": date.iloc[i],
        "url": url.iloc[i]
    })
pprint.pprint(dependencies[:5])

[{'date': datetime.datetime(2006, 7, 17, 0, 0),
  'library': 'activation-1.0.2.jar',
  'url': 'https://mvnrepository.com/artifact/javax.xml.bind/activation',
  'vulnerabilities': nan},
 {'date': nan, 'library': nan, 'url': '中菲', 'vulnerabilities': nan},
 {'date': datetime.datetime(2007, 1, 14, 0, 0),
  'library': 'antlr-3.0ea8.jar',
  'url': 'https://mvnrepository.com/artifact/antlr/antlr',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2007, 1, 14, 0, 0),
  'library': 'antlr-3.0ea8.jar',
  'url': 'https://mvnrepository.com/artifact/antlr/antlr',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2015, 12, 28, 0, 0),
  'library': 'aopalliance-1.0.jar',
  'url': 'https://mvnrepository.com/artifact/aopalliance/aopalliance',
  'vulnerabilities': nan}]



### step 2: loop through dependencies and scrap data from website

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

first_rows_of_version_table = []
for i, dep in enumerate(dependencies[:]):
    print(f"================== {i + 1} / {len(dependencies)} ==================")
    driver = webdriver.Chrome()
    url = dep["url"]
    start_time = time.perf_counter()
    try:
        print(f"scraping: {url}")
        driver.get(url)

        table = driver.find_element(By.CLASS_NAME, 'versions')
        thead = table.find_element(By.TAG_NAME, 'thead')
        tr = thead.find_element(By.TAG_NAME, "tr")
        ths = tr.find_elements(By.TAG_NAME, "th")
        
        headers= []
        for th in ths:
            headers.append(th.text)
        
        tbody = table.find_element(By.TAG_NAME, 'tbody')
        first_row = tbody.find_element(By.TAG_NAME, 'tr')
        cells = first_row.find_elements(By.TAG_NAME, 'td')

        rowData = {}
        for i, cell in enumerate(cells):
            try:
                x = cell.text
            except:
                x = ""
            rowData[headers[i]] =  x
        
        driver.close()
    except Exception as e:
        # invalid url
        print(f"Invalid url: {url}")
        rowData = {
            "Version" : "",
            "Vulnerabilities" : "",
            "Date" : ""
        }
        


    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(rowData)
    first_rows_of_version_table.append(rowData)
    print(f"Elapsed time: {elapsed_time} seconds")
    
    
driver.quit()

================== 1 / 94 ==================
scraping: https://mvnrepository.com/artifact/javax.xml.bind/activation
{'Version': '1.0.2', 'Vulnerabilities': '', 'Repository': 'Central', 'Usages': '42', 'Date': 'Jul 17, 2006'}
Elapsed time: 84.07735012518242 seconds
================== 2 / 94 ==================
scraping: 中菲
Invalid url: 中菲
{'Version': '', 'Vulnerabilities': '', 'Date': ''}
Elapsed time: 0.0029675420373678207 seconds
================== 3 / 94 ==================
scraping: https://mvnrepository.com/artifact/antlr/antlr
{'Version': '3.0ea8', 'Vulnerabilities': '', 'Repository': 'Central', 'Usages': '0', 'Date': 'Jan 14, 2007'}
Elapsed time: 82.70414187479764 seconds
================== 4 / 94 ==================
scraping: https://mvnrepository.com/artifact/antlr/antlr
{'Version': '3.0ea8', 'Vulnerabilities': '', 'Repository': 'Central', 'Usages': '0', 'Date': 'Jan 14, 2007'}
Elapsed time: 81.86165704205632 seconds
================== 5 / 94 ==================
scraping: https://m

In [31]:
pprint.pprint(first_rows_of_version_table)

[{'Date': 'Jul 17, 2006',
  'Repository': 'Central',
  'Usages': '42',
  'Version': '1.0.2',
  'Vulnerabilities': ''},
 {'Date': '', 'Version': '', 'Vulnerabilities': ''},
 {'Date': 'Jan 14, 2007',
  'Repository': 'Central',
  'Usages': '0',
  'Version': '3.0ea8',
  'Vulnerabilities': ''},
 {'Date': 'Jan 14, 2007',
  'Repository': 'Central',
  'Usages': '0',
  'Version': '3.0ea8',
  'Vulnerabilities': ''},
 {'Date': 'Dec 28, 2015',
  'Repository': 'Central',
  'Usages': '946',
  'Version': '1.0',
  'Vulnerabilities': ''},
 {'': '9.7.x',
  'Date': 'Mar 23, 2024',
  'Repository': 'Central',
  'Usages': '199',
  'Version': '9.7',
  'Vulnerabilities': ''},
 {'': '9.7.x',
  'Date': 'Mar 23, 2024',
  'Repository': 'Central',
  'Usages': '84',
  'Version': '9.7',
  'Vulnerabilities': ''},
 {'': '3.3.x',
  'Date': 'Aug 12, 2019',
  'Repository': 'Central',
  'Usages': '222',
  'Version': '3.3.0',
  'Vulnerabilities': ''},
 {'Date': 'Jun 14, 2023',
  'Repository': 'Central',
  'Usages': '0',
  

### step 3: organize data and parse to desired format

In [32]:
# Define formatting methods
import math
from datetime import datetime
def formatLibrary(libName, versionNumber):
    if (not libName or (type(libName) is float and math.isnan(libName))):
        return math.nan
    libNameList =libName.split('-')
    extension = libNameList[-1].split('.')[-1]
    return "".join(libNameList[:-1]) + "-" + versionNumber + "." + extension

def formatVulnerability(vulnerabilitiesString):
    if (not vulnerabilitiesString or (type(vulnerabilitiesString) is float and math.isnan(vulnerabilitiesString))):
        return math.nan
    return vulnerabilitiesString.split()[0]

def formatDate(dateString):
    if (not dateString or (type(dateString) is float and math.isnan(dateString))):
        return math.nan
    date_format = "%b %d, %Y"
    return datetime.strptime(dateString, date_format)

In [33]:
for i in range(len(dependencies[:])):
    row = first_rows_of_version_table[i]
    versionNumber = row["Version"]
    dateString = row["Date"]
    vulnerabilitiesString = row["Vulnerabilities"]
    
    formatedVersion = formatLibrary(dependencies[i]["library"], versionNumber)
    formatedVulnerability = formatVulnerability(vulnerabilitiesString)
    formatedDate = formatDate(dateString)
    
    dependencies[i]['date']= formatedDate
    dependencies[i]['library']= formatedVersion
    dependencies[i]['vulnerabilities']= formatedVulnerability

pprint.pprint(dependencies)

[{'date': datetime.datetime(2006, 7, 17, 0, 0),
  'library': 'activation-1.0.2.jar',
  'url': 'https://mvnrepository.com/artifact/javax.xml.bind/activation',
  'vulnerabilities': nan},
 {'date': nan, 'library': nan, 'url': '中菲', 'vulnerabilities': nan},
 {'date': datetime.datetime(2007, 1, 14, 0, 0),
  'library': 'antlr-3.0ea8.jar',
  'url': 'https://mvnrepository.com/artifact/antlr/antlr',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2007, 1, 14, 0, 0),
  'library': 'antlr-3.0ea8.jar',
  'url': 'https://mvnrepository.com/artifact/antlr/antlr',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2015, 12, 28, 0, 0),
  'library': 'aopalliance-1.0.jar',
  'url': 'https://mvnrepository.com/artifact/aopalliance/aopalliance',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2024, 3, 23, 0, 0),
  'library': 'asm-9.7.jar',
  'url': 'https://mvnrepository.com/artifact/org.ow2.asm/asm',
  'vulnerabilities': nan},
 {'date': datetime.datetime(2024, 3, 23, 0, 0),
  'library': 'a

### step 4: save data as a new sheet in excel

In [48]:
# write to a json file
import json

formatted_date = datetime.now().strftime('%Y%m%d')
filename = f'{formatted_date}_dependencies.json'

dep_copy = dependencies[:]
for dep in dep_copy:
    for k, value in dep.items():
        if isinstance(value, float) and math.isnan(value):
            dep[k] = ""
        if isinstance(value, datetime):
            dep[k] = value.isoformat()


# Writing the dictionary to a JSON file
with open(filename, 'w') as file:
    json.dump(dep_copy, file, indent=4, ensure_ascii=False,)


In [49]:
from openpyxl import load_workbook

# Create a new sheet in excel
workbook = load_workbook(excelPath)
sheet_to_duplicate = workbook.worksheets[0]

# Duplicate the sheet
new_sheet = workbook.copy_worksheet(sheet_to_duplicate)
formatted_date = datetime.now().strftime('%Y%m%d') # Format the date as 'YYYYMMDD'
new_sheet.title = "t" + str(formatted_date)

workbook._sheets.insert(0, workbook._sheets.pop(workbook.sheetnames.index(new_sheet.title)))

# Save the workbook
workbook.save(excelPath)

print(f"Duplicated '{sheet_to_duplicate.title}' as '{new_sheet.title}' in '{excelPath}'")

Duplicated '20240430' as 't20240703' in '/Users/yenshuohsu/DDSC/MavenWebScrapping/永豐.保管系統.Lib元件版本清單.20240630.xlsx'


In [50]:
headers = ['Date', 'Library', 'URL', 'Vulnerabilities']

# Write data rows
for row_idx, row_data in enumerate(dependencies, start=3):
    new_sheet.cell(row=row_idx, column=6, value=row_data.get('library'))
    new_sheet.cell(row=row_idx, column=7, value=row_data.get('vulnerabilities'))
    new_sheet.cell(row=row_idx, column=9, value=row_data.get('date'))
workbook.save(excelPath)